In [74]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import time
import json
import pickle

In [ ]:
Rating:
Archive Warning:
#check for category (optional)
Category:
#check for fandom
Fandom:
#check for relationship (optional)
Relationship:
#check for character
Character:
#check for additional tags (optional)
Additional Tags:
#check for series (optional)
Series:
#check for collections (optional)
Collections:
#check for stats
Stats:
    


In [75]:
soup_0005 = BeautifulSoup(open('Fanfic_HTML/0005.html'), "html.parser")
soup_13Heirs = BeautifulSoup(open('Fanfic_HTML/13 Heirs.html'), "html.parser")
soup_WizardComes = BeautifulSoup(open('Fanfic_HTML/In Which A Wizard Comes.html'), "html.parser")

In [78]:
def storyElements(soup_obj):
    local_classTags = soup_obj.find(class_="tags")
    i = 0
    for element in local_classTags:
        print(i)
        print(element)
        i += 1

In [79]:
storyElements(soup_0005)
storyElements(soup_13Heirs)
storyElements(soup_WizardComes)

0


1
<dt>Rating:</dt>
2


3
<dd><a href="http://archiveofourown.org/tags/General%20Audiences">General Audiences</a></dd>
4


5
<dt>Archive Warning:</dt>
6


7
<dd><a href="http://archiveofourown.org/tags/Choose%20Not%20To%20Use%20Archive%20Warnings">Choose Not To Use Archive Warnings</a></dd>
8


9
<dt>Category:</dt>
10


11
<dd><a href="http://archiveofourown.org/tags/Gen">Gen</a></dd>
12


13
<dt>Fandom:</dt>
14


15
<dd><a href="http://archiveofourown.org/tags/The%20Umbrella%20Academy%20(TV)">The Umbrella Academy (TV)</a>, <a href="http://archiveofourown.org/tags/The%20Umbrella%20Academy%20(Comics)">The Umbrella Academy (Comics)</a></dd>
16


17
<dt>Character:</dt>
18


19
<dd><a href="http://archiveofourown.org/tags/Number%20Five%20%7C%20The%20Boy%20(Umbrella%20Academy)">Number Five | The Boy (Umbrella Academy)</a></dd>
20


21
<dt>Additional Tags:</dt>
22


23
<dd><a href="http://archiveofourown.org/tags/Eating%20Disorders">Eating Disorders</a>, <a href="http://archiveofourown.or

In [203]:
def extractStoryRating(classTags):
    #code to extract story rating
    i = 0
    for element in classTags:
        i += 1
        if i == 4:
            story_rating = str(element.find('a').string)
        elif i > 4:
            break
            
    return story_rating

def extractArchiveWarning(classTags):
    i = 0
    for element in classTags:
        i += 1
        if i == 8:
            archiveWarning = str(element.find('a').string)
        elif i > 8:
            break
            
    return archiveWarning

def extractCategory(classTags):
    #code to extract category
    i = 0
    for element in classTags:
        i += 1
        if i == 12:
            story_category = str(element.find('a').string)
        elif i > 12:
            break

    return story_category

def extractFandoms(classTags):
    #code to extract fandoms
    i = 0
    j = 0
    fandom_text = ""
    for element in classTags:
        i += 1
        if str(element.string) == "Fandom:":
            j = 2 + i
        elif i == j:
            raw_fandom_text = element.find_all('a')
            for obj in raw_fandom_text:
                fandom_text += str(obj.string) + ', '
            fandom_text = fandom_text[:-2]

    return fandom_text

def extractRelationships(element):
    #code to extract relationships
    i = 0
    j = 0
    story_relationships = ""
    for element in classTags:
        i += 1
        if str(element.string) == "Relationship:":
            j = 2 + i
        elif i == j:
            raw_relationships = element.find_all('a')
            for obj in raw_relationships:
                story_relationships += str(obj.string) + ', '
            story_relationships = story_relationships[:-2]
            
    return story_relationships

def extractCharacters(classTags):
    #code to extract characters
    i = 0
    j = 0
    characters = ""
    for element in classTags:
        i += 1
        if str(element.string) == "Character:":
            j = 2 + i
        elif i == j:
            raw_characters = element.find_all('a')
            for obj in raw_characters:
                characters += str(obj.string) + ', '
            characters = characters[:-2]
            
    return characters

def extractAddTags(classTags):
    #code to extract tags
    i = 0
    j = 0
    additionalTags_text = ""
    for element in classTags:
        i += 1
        if str(element.string) == "Additional Tags:":
            j = 2 + i
        elif i == j:
            additionalTags = element.find_all('a')
            for obj in additionalTags:
                additionalTags_text += str(obj.string) + ', '
            additionalTags_text = additionalTags_text[:-2]

    return additionalTags_text

def extractSeries(classTags):
    #code to extract publication stats
    i = 0
    j = 0
    for element in classTags:
        i += 1
        if str(element.string) == "Series:":
            j = 2 + i
        elif i == j:
            series = str(element.find('a').string)

    return series

def extractCollections(classTags):
    #code to extract publication stats
    i = 0
    j = 0
    collections = ""
    for element in classTags:
        i += 1
        if str(element.string) == "Collections:":
            j = 2 + i
        elif i == j:
            collections = str(element.find('a').string)

    return collections

def extractPubStats(classTags):
    #code to extract publication stats
    i = 0
    j = 0
    for element in classTags:
        i += 1
        if str(element.string) == "Stats:":
            j = 2 + i
        elif i == j:
            pub_stat = str(element.string).strip()
            story_publish_date = pub_stat[11:21]
            story_word_count = pub_stat[37:40]
    
    return story_publish_date, story_word_count

In [204]:
tagsClass = soup_0005.find(class_="tags")
collections = extractCollections(tagsClass)

In [205]:
collections

'Bad Things Happen Bingo'

In [210]:
def extractTags(soup_obj):
    #code to extract metadata (fandom, characters, tags, warning, etc.)
    i = 0
    fandom_text = ""
    story_character = np.nan
    relationship = np.nan
    addTags = np.nan
    archiveWarning = np.nan
    category = np.nan
    fandoms = np.nan
    pubDate = np.nan
    wordCount = np.nan
    tagsClass = soup_obj.find(class_="tags")
    for element in tagsClass:
        if str(element.string) == "Rating:":
            rating = extractStoryRating(tagsClass)
            
        elif str(element.string) == "Archive Warning:":
            archiveWarning = extractArchiveWarning(tagsClass)
            
        elif str(element.string) == "Category:":
            category = extractCategory(tagsClass)
            
        elif str(element.string) == "Fandom:":
            fandom_text = extractFandoms(tagsClass)
        
        elif str(element.string) == "Relationship:":
            relationship = extractRelationships(tagsClass)
            
        elif str(element.string) == "Character:":
            story_character = extractCharacters(tagsClass)
            
        elif str(element.string) == "Additional Tags:":
            addTags = extractAddTags(tagsClass)
            
        elif str(element.string) == "Series:":
            series = extractSeries(tagsClass)
            
        elif str(element.string) == "Collections:":
            collections = extractCollections(tagsClass)
            
        elif str(element.string) == "Stats:":
            pubDate, wordCount = extractPubStats(tagsClass)
            
        i += 1
    
    local_tag_dict = {'rating' : rating,
                'archiveWarnings': archiveWarning,
                'category' : category,
                'fandom' : fandom_text,
                'relationships' : relationship,
                'character': story_character,
                'additionalTags': addTags,  
                'series': series, 
                'collections': collections,
                'pub_date' : pubDate, 
                'word_count' : wordCount
               }
        
    return local_tag_dict

In [211]:
tag_dict = extractTags(soup_0005)
tag_dict

{'rating': 'General Audiences',
 'archiveWarnings': 'Choose Not To Use Archive Warnings',
 'category': 'Gen',
 'fandom': 'The Umbrella Academy (TV), The Umbrella Academy (Comics)',
 'relationships': nan,
 'character': 'Number Five | The Boy (Umbrella Academy)',
 'additionalTags': 'Eating Disorders, Eating Disorder Not Otherwise Specified, Implied/Referenced Alcohol Abuse/Alcoholism, no beta we die like men',
 'series': 'Angsty Headcanon Ficlets',
 'collections': 'Bad Things Happen Bingo',
 'pub_date': '2019-04-28',
 'word_count': '327'}

In [207]:
def extractAuthor(soup_obj):
    #code to extract author and fandom info
    for info in soup_obj.title:
        author = str(info).split('-')[1]
        author = author.strip()
    return author
    

def extractTitleStory(soup_obj):
    #code to extract title and story text
    i = 0
    story_text = ""
    for element in soup_obj.find(id="chapters"):
        if i == 1:
            title = element.string
        elif i == 3:
            story_ptag_text = element.find_all('p')
            for obj in story_ptag_text:
                if obj.string == None:
                    continue
                else:
                    story_text += str(obj.string) + " "
        i += 1
    return title, story_text
        

def extractStoryURL(soup_obj): 
    #code to extract story URL
    i = 0
    for element in soup_obj.find(class_="message"):
        i += 1
        if i == 7:
            url = element['href']

        else:
            continue
    return url

def createAO3dict(filelist):
    i = 0
    ao3_dict = {}
    for element in filelist:
        soup = BeautifulSoup(open(element), "html.parser")
        url = extractStoryURL(soup)
        story_text = extractTitleStory(soup)
        tag_dict = extractTags(soup) #, story_series_name, story_collections
        author = extractAuthor(soup)
        title, story_text = extractTitleStory(soup)
        
        story_dict = {'title': title,
                      'author': author,
                      'story_text': story_text, 
                      'url': url,
                     }
        story_dict.update(tag_dict)
        
        ao3_dict[i] = story_dict
        i += 1
    return ao3_dict

In [178]:
fanfic = ['Fanfic_HTML/13 Heirs.html']
fanfics = ['Fanfic_HTML/13 Heirs.html', 'Fanfic_HTML/In Which A Wizard Comes.html', 'Fanfic_HTML/Sweet Basil and Italian.html', 
           'Fanfic_HTML/5 6 7 The Umbrella.html', 'Fanfic_HTML/In Which A Wizard Is.html', 'Fanfic_HTML/Swine and Forests.html',
           'Fanfic_HTML/50 Truths of the.html', 'Fanfic_HTML/In Which Calcifer Meets.html', 'Fanfic_HTML/Symbol of Reform.html',
           'Fanfic_HTML/A Bakers Proposal.html', 'Fanfic_HTML/In Which Howl Attracts.html', 'Fanfic_HTML/Sympathy.html']

In [181]:
ao3_dictionary = createAO3dict(fanfics)

In [182]:
ao3_dictionary

{0: {'title': '13 Heirs',
  'author': 'mayachain',
  'story_text': 'Sophie will never admit to the relief she feels when she finds out she won’t give Nate another son, nor to the prayers of thanks she leaves in Church when little Jemma turns out healthy despite the odds Sophie’s own age lay against it. Sophie and Nate’s daughter is going to grow up the most protected little girl in the universe. Only Olivia Sterling is perhaps similarly protected, but Jemma is the one who’ll have Eliot and Alec and Parker as her godparents and by virtue of their and her parents’ combined skills she will approach majority not only safe and secure but also capable of protecting herself and others as well as self-sufficient.  If the five of them do their jobs right she will never be forced to actually use what she knows for anything but fun. * Parker doesn’t want kids. She doesn’t want the change to her body, she doesn’t want to be slowed down, and no matter how much Alec and Eliot have come to love her, 

In [183]:
ao3_df = pd.DataFrame.from_dict(ao3_dictionary, orient="index")
ao3_df

title            author  \
0                                            13 Heirs         mayachain   
1                  In Which A Wizard Comes Home Drunk     bobblemonkey2   
2                     Sweet Basil and Italian Parsley       jenna_thorn   
3                        5 6 7 | The Umbrella Academy    AthenaPantheon   
4       In Which A Wizard Is Injured At A Rugby Match     bobblemonkey2   
5                                   Swine and Forests     Sour_Idealist   
6   50 Truths of the Umbrella Academy, by Allyson ...  GunpowderGelatin   
7                 In Which Calcifer Meets His Destiny         Untherius   
8                                    Symbol of Reform     cherryfeather   
9                                  A Baker's Proposal           hanluvr   
10  In Which Howl Attracts Customers and Sophie Pl...     bobblemonkey2   
11                                           Sympathy             Tibby   

                                           story_text  \
0   Sophie will never admit to the relief she feel...   
1   A clatter of pots and pans awoke Sophie in the...   
2   The thing is, Parker has no respect for the li...   
3   When her brothers were alive, Seven wasn’t alo...   
4   ‘Owwwww!’ Howl stomped in through the front do...   
5   “There’s a lot of magic around here, certainly...   
6   50 Truths of the Umbrella Academy - By Allyson...   
7   Calcifer floated in a state of weightlessness....   
8   He sets down his bottle with the other empties...   
9   I flee to the woods that have always been my r...   
10  Howl had been quite glum ever since the Witch ...   
11  Hunter rarely slept, but she knew it was essen...   

                                          url                 rating  \
0    http://archiveofourown.org/works/3284888      General Audiences   
1   http://archiveofourown.org/works/24759217      General Audiences   
2    http://archiveofourown.org/works/6747331  Teen And Up Audiences   
3   http://archiveofourown.org/works/25230484              Not Rated   
4   http://archiveofourown.org/works/24504826      General Audiences   
5     http://archiveofourown.org/works/222260      General Audiences   
6   http://archiveofourown.org/works/18002909  Teen And Up Audiences   
7     http://archiveofourown.org/works/301901      General Audiences   
8     http://archiveofourown.org/works/407223  Teen And Up Audiences   
9    http://archiveofourown.org/works/2657978      General Audiences   
10  http://archiveofourown.org/works/25864924      General Audiences   
11     http://archiveofourown.org/works/48750      General Audiences   

                       archiveWarnings category  \
0            No Archive Warnings Apply    Multi   
1            No Archive Warnings Apply      F/M   
2            No Archive Warnings Apply    Multi   
3   Choose Not To Use Archive Warnings      NaN   
4            No Archive Warnings Apply      F/M   
5            No Archive Warnings Apply      F/M   
6   Choose Not To Use Archive Warnings    Multi   
7            No Archive Warnings Apply      F/M   
8            No Archive Warnings Apply      F/M   
9            No Archive Warnings Apply      F/M   
10           No Archive Warnings Apply      F/M   
11           No Archive Warnings Apply      F/F   

                                               fandom  \
0                                            Leverage   
1   Howl’s Moving Castle, Howl no Ugoku Shiro | Ho...   
2                                            Leverage   
3   The Umbrella Academy (TV), The Umbrella Academ...   
4   Howl's Moving Castle - All Media Types, Howl S...   
5   Howl Series - Diana Wynne Jones, Enchanted For...   
6   The Umbrella Academy (TV), The Umbrella Academ...   
7                     Howl Series - Diana Wynne Jones   
8              Hunger Games Trilogy - Suzanne Collins   
9   Hunger Games Trilogy - Suzanne Collins, The Hu...   
10  Howl no Ugoku Shiro | Howl's Moving Castle, Ho...   
11                                Neverwhere 